# Módulo 4: Bases de datos - Ejercicio de evaluación

## 1. Creamos el comando para crear la base de datos y sus tablas en el archivo SQL:

In [1]:
import mysql.connector as con
import random
from datetime import datetime, timedelta

# Configuración de la conexión a MySQL
# He definido una función para establecer y retornar la conexión con la base de datos MySQL.
def get_connection():
    return con.connect(
        host="localhost",
        port="3306",
        user="root",
        password="admin",
        database="supermercado"
    )
    
# He creado una función para verificar si la base de datos 'supermercado' existe y, en caso contrario, crearla.
def create_database():
    connection = con.connect(
        host="localhost",
        port="3306",
        user="root",
        password="admin"
    )
    cursor = connection.cursor()
    cursor.execute("SHOW DATABASES;")  # Obtengo las bases de datos existentes
    databases = [db[0] for db in cursor.fetchall()]  # Genero una lista de bases de datos
    if "supermercado" not in databases:  # Si la base de datos no existe, la creo
        cursor.execute("CREATE DATABASE supermercado DEFAULT CHARACTER SET utf8;")
        print("Base de datos creada exitosamente.")
    else:
        print("La base de datos ya existe.")
    cursor.close()
    connection.close()

# He creado una función que define y crea las tablas necesarias en la base de datos 'supermercado'.
def create_tables():
    connection = get_connection()
    cursor = connection.cursor()
    
    # Defino los esquemas de cada tabla con las relaciones necesarias.
    tables = {
        "tiendas": """
            CREATE TABLE IF NOT EXISTS tiendas (
                id_tienda INT AUTO_INCREMENT PRIMARY KEY,
                nombre_tienda VARCHAR(100) NOT NULL,
                direccion VARCHAR(255) NOT NULL,
                ciudad VARCHAR(100) NOT NULL
            );
        """,
        "empleados": """
            CREATE TABLE IF NOT EXISTS empleados (
                id_empleado INT AUTO_INCREMENT PRIMARY KEY,
                nombre_empleado VARCHAR(100) NOT NULL,
                puesto ENUM('Cajero', 'Gerente', 'Reponedor', 'Vendedor') NOT NULL,
                id_tienda INT,
                FOREIGN KEY (id_tienda) REFERENCES tiendas(id_tienda) ON DELETE SET NULL
            );
        """,
        "clientes": """
            CREATE TABLE IF NOT EXISTS clientes (
                id_cliente INT AUTO_INCREMENT PRIMARY KEY,
                nombre_cliente VARCHAR(100) NOT NULL,
                email VARCHAR(150) UNIQUE NOT NULL,
                telefono VARCHAR(15) NOT NULL,
                direccion VARCHAR(255) NOT NULL
            );
        """,
        "categorias": """
            CREATE TABLE IF NOT EXISTS categorias (
                id_categoria INT AUTO_INCREMENT PRIMARY KEY,
                nombre_categoria VARCHAR(100) NOT NULL
            );
        """,
        "productos": """
            CREATE TABLE IF NOT EXISTS productos (
                id_producto INT AUTO_INCREMENT PRIMARY KEY,
                nombre_producto VARCHAR(100) NOT NULL,
                precio DECIMAL(10,2) NOT NULL,
                stock INT NOT NULL,
                id_categoria INT,
                FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria) ON DELETE SET NULL
            );
        """,
        "ordenes": """
            CREATE TABLE IF NOT EXISTS ordenes (
                id_orden INT AUTO_INCREMENT PRIMARY KEY,
                id_cliente INT NOT NULL,
                id_empleado INT NOT NULL,
                fecha_orden DATETIME DEFAULT CURRENT_TIMESTAMP,
                metodo_pago ENUM('Tarjeta', 'Efectivo') NOT NULL,
                FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente) ON DELETE CASCADE,
                FOREIGN KEY (id_empleado) REFERENCES empleados(id_empleado) ON DELETE CASCADE
            );
        """,
        "detalle_orden": """
            CREATE TABLE IF NOT EXISTS detalle_orden (
                id_detalle INT AUTO_INCREMENT PRIMARY KEY,
                id_orden INT NOT NULL,
                id_producto INT NOT NULL,
                cantidad INT NOT NULL,
                precio_unitario DECIMAL(10,2) NOT NULL,
                descuento DECIMAL(5,2) DEFAULT 0.00,
                FOREIGN KEY (id_orden) REFERENCES ordenes(id_orden) ON DELETE CASCADE,
                FOREIGN KEY (id_producto) REFERENCES productos(id_producto) ON DELETE CASCADE
            );
        """
    }

    # Itero sobre el diccionario de tablas y ejecuto las consultas para crearlas.
    for table, query in tables.items():
        cursor.execute(query)
    connection.commit()
    cursor.close()
    connection.close()
    print("Tablas creadas correctamente.")

 ## 2: Generar datos demo desde Python

In [25]:
# He definido una función para insertar datos en cualquier tabla en lotes para mejorar la eficiencia.
def insert_data(table, columns, values, batch_size=30000):
    connection = get_connection()
    cursor = connection.cursor()
    try:
        placeholders = ", ".join(["%s"] * len(columns))
        column_names = ", ".join(columns)
        query = f"INSERT INTO {table} ({column_names}) VALUES ({placeholders});"
        
        # Dividir los datos en lotes (batches)
        for i in range(0, len(values), batch_size):
            batch = values[i:i+batch_size]
            cursor.executemany(query, batch)
            connection.commit()
        print(f"Datos insertados en {table} correctamente.")
    except Exception as e:
        connection.rollback()
        print(f"Error en {table}: {e}")
    finally:
        cursor.close()
        connection.close()

# Antes de insertar datos iré metiendo todas las variables necesarias para insertar los datos de cada tabla:

calles = ["Avenida Al-Azhar", "Calle Medina", "Boulevard de la Meca", "Avenida de los Mártires", "Calle de la Paz",
        "Gran Vía", "Calle del Prado", "Avenida de Mayo", "Calle de Alcalá", "Paseo de la Castellana",
        "Oxford Street", "Baker Street", "Downing Street", "Piccadilly Circus", "Fleet Street",
        "Avenida Shibuya", "Calle Ginza", "Paseo de Roppongi", "Calle Akihabara", "Avenida Shinjuku",
        "Via Roma", "Corso Vittorio Emanuele", "Piazza San Marco", "Via della Conciliazione", "Viale Monte Nero"]
    
ciudades = ["La Meca", "Medina", "Bagdad", "El Cairo", "Damasco", "Estambul", "Kuala Lumpur", "Riad", "Casablanca", "Alejandría",
        "Madrid", "Barcelona", "Sevilla", "Valencia", "Granada", "Zaragoza", "Bilbao", "Murcia", "Málaga", "Palma de Mallorca",
        "Londres", "Edimburgo", "Manchester", "Birmingham", "Liverpool", "Nottingham", "Glasgow", "Leeds", "Bristol", "Cardiff",
        "Tokio", "Osaka", "Kioto", "Hiroshima", "Nagoya", "Sapporo", "Fukuoka", "Kobe", "Yokohama", "Sendai",
        "Roma", "Milán", "Nápoles", "Turín", "Palermo", "Bolonia", "Florencia", "Venecia", "Catania", "Verona"]
    
direcciones = ['Norte', 'Sur', 'Este', 'Oeste']

nombres = ["Mohammed", "Fatima", "Zainab", "Ali", "Khadija", "Omar", "Amina", "Hassan", "Rashid", "Yusuf",
        "Santiago", "Ana", "José", "María", "Luis", "Sofía", "Miguel", "Laura", "Juan", "Isabel",
        "James", "Mary", "Joseph", "Emily", "Michael", "Sarah", "David", "Emma", "William", "Charlotte",
        "Taro", "Yuki", "Jotaro", "Sakura", "Gintoki", "Kaito", "Riko", "Yui", "Ren", "Aoi",
        "Giovanni", "Giorno", "Bruno", "Giulia", "Marco", "Francesca", "Luca", "Alessandra", "Matteo", "Elena"]

apellidos = [
        "Ahmed", "Hassan", "Avdol", "Omar", "Ibrahim", "Yusuf", "Khan", "Ali", "Farooq", "Jabbar",
        "García", "Martínez", "López", "González", "de Cervantes", "Fernández", "Pérez", "Sánchez", "Ramírez", "de Compostela",
        "Smith", "Johnson", "Williams", "Jones", "Brown", "Joestar", "Miller", "Wilson", "Moore", "Taylor",
        "Takahashi", "Tanaka", "Yamamoto", "Kobayashi", "Sato", "Kato", "Kujo", "Nakamura", "Sakata", "Yoshida",
        "Rossi", "Bianchi", "Romano", "Ferrari", "Giordano", "Ricci", "Esposito", "Giovanna", "Lombardi", "Moretti"]

categorias = ["Lácteos", "Carnes", "Frutas", "Verduras", "Bebidas", "Snacks", "Limpieza", "Medicina", "Panadería", "Tecnología"]

# Diccionario con productos específicos para cada categoría
productos_por_categoria = {
    "Lácteos": ["Leche", "Yogur", "Queso", "Mantequilla"],
    "Carnes": ["Pollo", "Ternera", "Cerdo", "Pavo"],
    "Frutas": ["Manzana", "Banana", "Naranja", "Pera"],
    "Verduras": ["Lechuga", "Zanahoria", "Tomate", "Cebolla"],
    "Bebidas": ["Agua", "Jugo", "Refresco", "Cerveza"],
    "Snacks": ["Galletas", "Chocolate", "Chips", "Frutos secos"],
    "Limpieza": ["Detergente", "Jabón", "Esponja", "Desinfectante"],
    "Medicina": ["Paracetamol", "Ibuprofeno", "Jarabe", "Vitamina C"],
    "Panadería": ["Pan", "Croissant", "Baguette", "Dona"],
    "Tecnología": ["Teléfono", "Laptop", "Auriculares", "Monitor"]
}

email = ["@hotmail.com", "@gmail.com", "@outlook.com", "@yahoo.com", "@icloud.com"]

num_tiendas = random.randint(5, 10)  # Generar entre 5 y 10 tiendas
def generate_tiendas():
    
    # Generar datos de las tiendas
    tienda_data = []
    for i in range(1, num_tiendas + 1):
        tipo_tienda = random.choice(['Super', 'Tienda'])
        direccion = f"{random.choice(calles)} {random.randint(1, 1000)}"
        ciudad = random.choice(ciudades)
        nombre_tienda = f"{tipo_tienda} {random.choice(direcciones)} {random.choice(nombres)}"
        
        # Añadir los datos de la tienda a la lista
        tienda_data.append((nombre_tienda, direccion, ciudad))
    
    # Insertar los datos en la base de datos usando insert_data
    insert_data("tiendas", ["nombre_tienda", "direccion", "ciudad"], tienda_data)
    print(f"Se han generado {num_tiendas} tiendas.")
    
def generate_empleados():
    # Obtener el número de tiendas generadas
    num_tienda = random.randint(1, num_tiendas)  # Asumimos que el número de tiendas es entre 5 y 10 como en la función generate_tiendas
    
    # Genero datos aleatorios para la tabla 'empleados'.
    empleados = []
    for i in range(1, 21):  # Generar 20 empleados
        nombre_empleado = f"{random.choice(nombres)} {random.choice(apellidos)}"
        puesto = random.choice(['Cajero', 'Gerente', 'Reponedor', 'Vendedor'])
        
        # Asignar aleatoriamente una tienda de las existentes entre 1 y num_tiendas
        empleados.append((nombre_empleado, puesto, random.randint(1, num_tiendas)))  # Aleatorización correcta de la tienda
    
    # Insertar los datos de los empleados en la base de datos
    insert_data("empleados", ["nombre_empleado", "puesto", "id_tienda"], empleados)

def generate_clientes():
    # Genero datos aleatorios para la tabla 'clientes'.
    clientes = []
    
    for i in range(1, 2001):  # 2000 clientes
        nombre_cliente = f"{random.choice(nombres)}"
        apellidos_cliente = f"{random.choice(apellidos)}"
        nombre_completo = f"{nombre_cliente} {apellidos_cliente}"
        nombre_email=f"{random.choice(email)}"
        correo = f"{nombre_cliente}{apellidos_cliente}{random.randint(1, 9999)}{nombre_email}"
        telefono = f"{random.randint(600000000, 999999999)}"  # Números de teléfono ficticios
        direccion = f"{random.choice(calles)} {random.randint(1, 1000)}"
        clientes.append((nombre_completo, correo, telefono, direccion))
    insert_data("clientes", ["nombre_cliente", "email", "telefono", "direccion"], clientes)

def generate_categorias():
    # Genero los datos de las categorías
    categoria_data = []
    for i in range(1, 11):  # Generamos 10 categorías
        nombre_categoria = categorias[i-1]
        categoria_data.append((i, nombre_categoria))
    
    # Insertar las categorías en la base de datos
    insert_data("categorias", ["id_categoria", "nombre_categoria"], categoria_data)

def generate_productos():
    productos_generados = []  # Lista para almacenar los productos generados
    
    for categoria_id, categoria_nombre in enumerate(categorias, start=1):  # Enumeramos desde 1
        productos = productos_por_categoria[categoria_nombre]  # Obtiene los productos de esa categoría
        
        for nombre_producto in productos:
            precio = round(random.uniform(0.5, 50.0), 2)  # Genera precio aleatorio
            stock = random.randint(0, 500)  # Genera stock aleatorio
            productos_generados.append((nombre_producto, precio, stock, categoria_id))
    
    # Insertar los productos en la base de datos
    insert_data("productos", ["nombre_producto", "precio", "stock", "id_categoria"], productos_generados)
    
def generate_ordenes():
    # Genera datos aleatorios para la tabla 'ordenes'.
    ordenes = []
    clientes_ids = [i for i in range(1, 2001)]  # 2000 clientes insertados previamente
    empleados_ids = [i for i in range(1, 21)]  # 20 empleados insertados previamente

    for i in range(1, 10001):
        id_cliente = random.choice(clientes_ids)
        id_empleado = random.choice(empleados_ids)
        fecha_orden = generate_random_dates("2024-01-01", "2025-12-31", 1)[0]
        metodo_pago = random.choice(['Tarjeta', 'Efectivo'])
        ordenes.append((id_cliente, id_empleado, fecha_orden, metodo_pago))

    insert_data("ordenes", ["id_cliente", "id_empleado", "fecha_orden", "metodo_pago"], ordenes)

def generate_detalle_orden():
    # Genera datos aleatorios para la tabla 'detalle_orden'.
    detalle_orden = []
    productos_ids = [i for i in range(1, 41)]
    ordenes_ids = [i for i in range(1, 10001)]

    for i in range(1, 30001):
        id_orden = random.choice(ordenes_ids)
        id_producto = random.choice(productos_ids)
        cantidad = random.randint(1, 20)
        precio_unitario = round(random.uniform(0.5, 50.0), 2)
        descuento = random.choice([0.00, round(random.uniform(0.5, 5.0), 2)])
        detalle_orden.append((id_orden, id_producto, cantidad, precio_unitario, descuento))

    insert_data("detalle_orden", ["id_orden", "id_producto", "cantidad", "precio_unitario", "descuento"], detalle_orden)

def generate_random_dates(start_date, end_date, num_dates):
    # Genera fechas aleatorias entre un rango de fechas.
    start_timestamp = datetime.strptime(start_date, "%Y-%m-%d")
    end_timestamp = datetime.strptime(end_date, "%Y-%m-%d")
    dates = []
    for _ in range(num_dates):
        random_date = start_timestamp + (end_timestamp - start_timestamp) * random.random()
        dates.append(random_date)
    return dates

# Defino una función para eliminar todos los registros de las tablas antes de insertar nuevos datos.
def clear_tables():
    tables = ["tiendas", "empleados", "categorias", "productos", "clientes", "ordenes", "detalle_orden"]
    connection = get_connection()
    cursor = connection.cursor()
    try:
        cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
        for table in tables:
            query = f"TRUNCATE TABLE {table};"
            cursor.execute(query)
        connection.commit()
        cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
    except Exception as e:
        connection.rollback()
        print(f"Error al limpiar las tablas: {e}")
    finally:
        cursor.close()
        connection.close()

# Resultado:

# Llamada para crear la base de datos y las tablas

create_database()
create_tables()
# Limpio las tablas para que se inserten bien los datos en caso de que haya datos existentes
clear_tables()

# Ahora comprobaré si se generan bien los datos
generate_tiendas()
generate_empleados()
generate_clientes()
generate_categorias()
generate_productos()
generate_ordenes()
generate_detalle_orden()

La base de datos ya existe.
Tablas creadas correctamente.
Datos insertados en tiendas correctamente.
Se han generado 6 tiendas.
Datos insertados en empleados correctamente.
Datos insertados en clientes correctamente.
Datos insertados en categorias correctamente.
Datos insertados en productos correctamente.
Datos insertados en ordenes correctamente.
Datos insertados en detalle_orden correctamente.


## 3: Consultas SQL

### 1 - Listado de órdenes con detalles de cliente y empleado:

```sql
SELECT 
    o.id_orden, 
    o.fecha_orden, 
    c.nombre_cliente, 
    e.nombre_empleado, 
    o.metodo_pago
FROM 
    supermercado.ordenes o
JOIN 
    supermercado.clientes c ON o.id_cliente = c.id_cliente
JOIN 
    supermercado.empleados e ON o.id_empleado = e.id_empleado;
```

### 2 - Productos con stock bajo (menor a 10):

```sql
SELECT 
    p.nombre_producto, 
    ca.nombre_categoria, 
    p.stock
FROM 
    supermercado.productos p
JOIN 
    supermercado.categorias ca ON p.id_categoria = ca.id_categoria
WHERE 
    p.stock < 10;
```

### 3 - Ventas totales por categoría:

```sql
SELECT 
    ca.nombre_categoria, 
    ROUND(SUM(do.cantidad * do.precio_unitario), 2) AS ventas_totales
FROM 
    supermercado.detalle_orden do
JOIN 
    supermercado.productos p ON do.id_producto = p.id_producto
JOIN 
    supermercado.categorias ca ON p.id_categoria = ca.id_categoria
GROUP BY 
    ca.id_categoria;
```

### 4 - Clientes con mayores gastos acumulados:

```sql
SELECT 
    c.nombre_cliente,
    SUM(do.cantidad * do.precio_unitario - IFNULL(do.descuento, 0)) AS gasto_total
FROM 
    supermercado.detalle_orden do
JOIN 
    supermercado.ordenes o ON do.id_orden = o.id_orden
JOIN 
    supermercado.clientes c ON o.id_cliente = c.id_cliente
GROUP BY 
    c.id_cliente
ORDER BY 
    gasto_total DESC;
```

### 5 - Empleados y número de órdenes gestionadas:

```sql
SELECT 
    e.nombre_empleado, 
    e.puesto, 
    COUNT(o.id_orden) AS num_ordenes
FROM 
    supermercado.empleados e
LEFT JOIN 
    supermercado.ordenes o ON e.id_empleado = o.id_empleado
GROUP BY 
    e.id_empleado;
```

### 6 - Órdenes filtradas por fecha y tienda:

```sql
SELECT 
    o.id_orden, 
    o.fecha_orden, 
    t.nombre_tienda, 
    c.nombre_cliente
FROM 
    supermercado.ordenes o
JOIN 
    supermercado.empleados e ON o.id_empleado = e.id_empleado
JOIN 
    supermercado.tiendas t ON e.id_tienda = t.id_tienda
JOIN 
    supermercado.clientes c ON o.id_cliente = c.id_cliente
WHERE 
    o.fecha_orden BETWEEN '2024-01-01' AND '2024-12-31'
    AND t.id_tienda = 1;
```

### 7 - Ranking de productos más vendidos en cada tienda:

```sql
SELECT 
    t.nombre_tienda, 
    p.nombre_producto, 
    SUM(do.cantidad) AS total_vendido
FROM 
    supermercado.detalle_orden do
JOIN 
    supermercado.ordenes o ON do.id_orden = o.id_orden
JOIN 
    supermercado.empleados e ON o.id_empleado = e.id_empleado
JOIN 
    supermercado.tiendas t ON e.id_tienda = t.id_tienda
JOIN 
    supermercado.productos p ON do.id_producto = p.id_producto
GROUP BY 
    t.id_tienda, p.id_producto
ORDER BY 
    t.id_tienda, total_vendido DESC
LIMIT 3;
```

### Opcional - Cliente que más productos ha comprado:

```sql
SELECT 
    c.nombre_cliente, 
    (SELECT SUM(d.cantidad) 
     FROM supermercado.detalle_orden AS d
     JOIN supermercado.ordenes AS o ON d.id_orden = o.id_orden
     WHERE o.id_cliente = c.id_cliente
    ) AS total_productos_comprados
FROM 
    supermercado.clientes AS c
ORDER BY 
    total_productos_comprados DESC
LIMIT 1;
```

cursor.execute('''
select * from customers;
''')